# Load Candles from Cache
This notebook demonstrates how to load previously downloaded candles data from the cache.

The cache stores candles data in Parquet format for efficient storage and retrieval.

In [ ]:
from core.data_sources.clob import CLOBDataSource
import warnings

warnings.filterwarnings("ignore")

# Initialize CLOBDataSource
clob = CLOBDataSource()

In [ ]:
# Load all cached candles data
clob.load_candles_cache()

# Display what's available in the cache
print(f"Total cached items: {len(clob.candles_cache)}")
print("\nAvailable cached data:")
for key, candles in clob.candles_cache.items():
    connector, pair, interval = key
    print(f"  - {connector} | {pair} | {interval} | {len(candles.data)} candles")

In [ ]:
# Access specific candles from cache
# Example: Get candles for specific connector, trading pair and interval
CONNECTOR_NAME = "binance_perpetual"
TRADING_PAIR = "BTC-USDT"
INTERVAL = "1m"

candles = clob.get_candles_from_cache(
    connector_name=CONNECTOR_NAME,
    trading_pair=TRADING_PAIR,
    interval=INTERVAL
)

if candles:
    print(f"Successfully loaded {len(candles.data)} candles for {TRADING_PAIR}")
    print(f"\nDate range: {candles.data.index.min()} to {candles.data.index.max()}")
else:
    print(f"No cached data found for {CONNECTOR_NAME} | {TRADING_PAIR} | {INTERVAL}")

In [ ]:
# Display candles data
if candles:
    print("\nFirst 5 candles:")
    display(candles.data.head())
    
    print("\nLast 5 candles:")
    display(candles.data.tail())

In [ ]:
# Plot the candles
if candles:
    candles.plot()

In [ ]:
# Working with multiple cached datasets
# Get all available candles and perform analysis

for key, candles_data in clob.candles_cache.items():
    connector, pair, interval = key
    df = candles_data.data
    
    # Calculate basic statistics
    avg_volume = df['volume'].mean()
    price_volatility = df['close'].pct_change().std()
    
    print(f"\n{pair} ({interval}):")
    print(f"  Average Volume: {avg_volume:,.2f}")
    print(f"  Price Volatility: {price_volatility:.4%}")
    print(f"  Price Range: ${df['low'].min():.4f} - ${df['high'].max():.4f}")

## Advanced Usage

The cache system automatically merges new data with existing cached data when you fetch candles. This means:
- If you request data that's already cached, it returns instantly from cache
- If you request data partially in cache, it only fetches the missing parts
- The cache is stored in Parquet format for efficient disk storage

In [ ]:
# Example: Fetch additional data that extends the cache
# This will check cache first and only download missing data, modify the days parameter to extend the range

import asyncio

async def fetch_extended_data():
    # This will merge with existing cache automatically
    extended_candles = await clob.get_candles_last_days(
        connector_name="binance_perpetual",
        trading_pair="1000SHIB-USDT",
        interval="1m",
        days=10  # Extend to 60 days if not already cached
    )
    return extended_candles

# Run the async function
extended_candles = await fetch_extended_data()
print(f"Total candles after extension: {len(extended_candles.data)}")